This is the third assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Again, please insert to code to your ApacheCouchDB based Cloudant instance below using the "Insert Code" function of Watson Studio (you've done this in Assignment 1 and 2 before)

Done, just execute all cells one after the other and you are done - just note that in the last one you must update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"


In [1]:
# The code was removed by Watson Studio for sharing.

Let's create a SparkSession object and put the Cloudant credentials into it

In [2]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .config("jsonstore.rdd.partitions", 1)\
    .getOrCreate()

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [3]:
df=spark.read.load('shake_classification', "com.cloudant.spark")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+--------+-----+-----+-----+--------------------+--------------------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|
+-----+--------+-----+-----+-----+--------------------+--------------------+
|    0|qwertyui|-0.01|-0.01|-0.01|204a988dc54034655...|1-d420fe4e0aecf9e...|
|    0|qwertyui|  0.0|  0.0|  0.0|204a988dc54034655...|1-79a425ac6fa2a93...|
|    0|qwertyui|-0.02|-0.02|-0.02|204a988dc54034655...|1-241442bb9333788...|
|    0|qwertyui| 0.03| 0.03| 0.03|204a988dc54034655...|1-e74f8da2535b668...|
|    0|qwertyui| 0.01| 0.01| 0.01|204a988dc54034655...|1-8b997db32f3eb67...|
|    0|qwertyui|-0.02|-0.02|-0.02|204a988dc54034655...|1-241442bb9333788...|
|    0|qwertyui| 0.01| 0.01| 0.01|204a988dc54034655...|1-8b997db32f3eb67...|
|    0|qwertyui| 0.02| 0.02| 0.02|204a988dc54034655...|1-1ab8e1f2d755c03...|
|    0|qwertyui|  0.0|  0.0|  0.0|204a988dc54034655...|1-79a425ac6fa2a93...|
|    0|qwertyui|-0.02|-0.02|-0.02|204a988dc54034655...|1-241442bb9333788...|

Let’s check if we have balanced classes – this means that we have roughly the same number of examples for each class we want to predict. This is important for classification but also helpful for clustering

In [4]:
spark.sql("SELECT count(class), class from df group by class").show()

+------------+-----+
|count(class)|class|
+------------+-----+
|        2298|    0|
+------------+-----+



Let's create a VectorAssembler which consumes columns X, Y and Z and produces a column “features”


In [5]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X","Y","Z"],
                                  outputCol="features")

Please insatiate a clustering algorithm from the SparkML package and assign it to the clust variable. Here we don’t need to take care of the “CLASS” column since we are in unsupervised learning mode – so let’s pretend to not even have the “CLASS” column for now – but it will become very handy later in assessing the clustering performance. PLEASE NOTE – IN REAL-WORLD SCENARIOS THERE IS NO CLASS COLUMN – THEREFORE YOU CAN’T ASSESS CLASSIFICATION PERFORMANCE USING THIS COLUMN 



In [6]:
from pyspark.ml.clustering import KMeans

clust = KMeans().setK(2).setSeed(1)

Let’s train...


In [7]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, clust])
model = pipeline.fit(df)

...and evaluate...

In [8]:
prediction = model.transform(df)
prediction.show()

+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|CLASS|SENSORID|    X|    Y|    Z|                 _id|                _rev|           features|prediction|
+-----+--------+-----+-----+-----+--------------------+--------------------+-------------------+----------+
|    0|qwertyui|-0.01|-0.01|-0.01|204a988dc54034655...|1-d420fe4e0aecf9e...|[-0.01,-0.01,-0.01]|         0|
|    0|qwertyui|  0.0|  0.0|  0.0|204a988dc54034655...|1-79a425ac6fa2a93...|          (3,[],[])|         0|
|    0|qwertyui|-0.02|-0.02|-0.02|204a988dc54034655...|1-241442bb9333788...|[-0.02,-0.02,-0.02]|         0|
|    0|qwertyui| 0.03| 0.03| 0.03|204a988dc54034655...|1-e74f8da2535b668...|   [0.03,0.03,0.03]|         0|
|    0|qwertyui| 0.01| 0.01| 0.01|204a988dc54034655...|1-8b997db32f3eb67...|   [0.01,0.01,0.01]|         0|
|    0|qwertyui|-0.02|-0.02|-0.02|204a988dc54034655...|1-241442bb9333788...|[-0.02,-0.02,-0.02]|         0|
|    0|qwertyui| 0.01| 0.01|

In [9]:
prediction.createOrReplaceTempView('prediction')
spark.sql('''
select max(correct)/max(total) as accuracy from (

    select sum(correct) as correct, count(correct) as total from (
        select case when class != prediction then 1 else 0 end as correct from prediction 
    ) 
    
    union
    
    select sum(correct) as correct, count(correct) as total from (
        select case when class = prediction then 1 else 0 end as correct from prediction 
    ) 
)
''').rdd.map(lambda row: row.accuracy).collect()[0]

0.8986074847693647

If you reached at least 55% of accuracy you are fine to submit your predictions to the grader. Otherwise please experiment with parameters setting to your clustering algorithm, use a different algorithm or just re-record your data and try to obtain. In case you are stuck, please use the Coursera Discussion Forum. Please note again – in a real-world scenario there is no way in doing this – since there is no class label in your data. Please have a look at this further reading on clustering performance evaluation https://en.wikipedia.org/wiki/Cluster_analysis#Evaluation_and_assessment


In [10]:
!rm -Rf a2_m3.json

In [11]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2018-12-31 00:18:35--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2018-12-31 00:18:36 (17.4 MB/s) - ‘rklib.py’ saved [2289/2289]



In [12]:
prediction= prediction.repartition(1)
prediction.write.json('a2_m3.json')

In [13]:
!zip -r a2_m3.json.zip a2_m3.json

  adding: a2_m3.json/ (stored 0%)
  adding: a2_m3.json/.part-00000-ed0d1b2c-0e06-49de-8566-52671521dcf1.json.crc (stored 0%)
  adding: a2_m3.json/_SUCCESS (stored 0%)
  adding: a2_m3.json/._SUCCESS.crc (stored 0%)
  adding: a2_m3.json/part-00000-ed0d1b2c-0e06-49de-8566-52671521dcf1.json (deflated 91%)


In [14]:
!base64 a2_m3.json.zip > a2_m3.json.zip.base64

In [15]:
from rklib import submit
key = "pPfm62VXEeiJOBL0dhxPkA"
part = "EOTMs"
email = "tommyhpratama@gmail.com"
secret = "eXeUhp7TcwM5XtGl"

with open('a2_m3.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"Cu6KW","id":"f_F-qCtuEei_fRLwaVDk3g~Cu6KW~6XsEcAzDEemMKRIFUZOhcA","courseId":"f_F-qCtuEei_fRLwaVDk3g"}],"paging":{},"linked":{}}
-------------------------
